## Imports

In [1]:
import cv2
import numpy as np
import time

## Obstacle Detection (Black) 

In [2]:
# Initialize variables for averaging
num_frames_average_black = 30   # Adjust this value
max_tracked_objects_black = 4

# Initializing values
blk_avg_count = 0
avg_cx_black = [0] * max_tracked_objects_black
avg_cy_black = [0] * max_tracked_objects_black
num_tracked_objects = 0
black_refs = []
    
def track_black (frame):
    global blk_avg_count, avg_cx_black, avg_cy_black, num_tracked_objects, black_refs, black_ref

    ## ----------Detecting Black Objects----------
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
    # Apply adaptive thresholding
    # . Pixels with intensity greater than 45 are set to 255 (white), and others are set to 0 (black). 
    # Lowering the threshold to detect darker colors
    _, thresh = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY_INV)

    # Apply morphological operations to reduce noise
    # Small kernel = track small objects, large kernel = more likely to smooth out details and may help in reducing noise or small variations
    kernel = np.ones((5, 5), np.uint8)  # creates a 5x5 square shaped kernel
    # Changing the iterations will help to improve the noise. 
    # Each iteration of the operation modifies the image, and the result becomes the input for the next iteration.
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=3)

    # ------------Detecting and averaging centroid----------
    # Find contours in the binary image
    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on area in descending order and count the number of tracked objects
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:max_tracked_objects_black]

    # Update the number of tracked objects
    num_tracked_objects = min(len(sorted_contours), max_tracked_objects_black)

    # Calculate the average centroid value
    for i, contour in enumerate(sorted_contours):

        M_black = cv2.moments(contour)
        if M_black["m00"] != 0:
            cx_black = int(M_black["m10"] / M_black["m00"])
            cy_black = int(M_black["m01"] / M_black["m00"])
        else:
            cx_black, cy_black = 0, 0

        if i < max_tracked_objects_black:
            avg_cx_black[i] += cx_black
            avg_cy_black[i] += cy_black

        # Draw a dot at the centroid
        cv2.circle(frame, (cx_black, cy_black), 2, (120, 0, 120), -1)

        # Draw the bounding box
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (120, 0, 120), 2)

         # Add the label "Obstacle" inside the bounding box
        label = "Obstacle".format(cx_black, cy_black)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

    # Counter for average centroid
    blk_avg_count += 1
    # print("Black counter: {}".format(blk_avg_count))

    if blk_avg_count == num_frames_average_black:
        for i in range(num_tracked_objects):
            avg_cx_black[i] = round(avg_cx_black[i] / num_frames_average_black, 2)
            avg_cy_black[i] = round(avg_cy_black[i] / num_frames_average_black, 2)
            print("Obstacle_Center {}: ({}, {})".format(i+1, avg_cx_black[i], avg_cy_black[i]))

        # Reset the accumulated values and counter for the next set of frames
        blk_avg_count = 0
        avg_cx_black = [0] * max_tracked_objects_black
        avg_cy_black = [0] * max_tracked_objects_black
            


    return frame

## Goal Detection (Blue)

In [3]:
# Initialize variables for averaging
num_frames_average_blue = 30   # Adjust this value
max_tracked_objects_blue = 1   # Adjust this value

# Initializing values
blue_avg_count = 0
avg_cx_blue = [0] * max_tracked_objects_blue
avg_cy_blue = [0] * max_tracked_objects_blue
num_tracked_objects_blue = 0

def track_blue(frame):
    global blue_avg_count, avg_cx_blue, avg_cy_blue, num_tracked_objects_blue

    ## ----------Detecting Blue Objects----------
    
    # Convert the frame from BGR to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the blue color
    # defined as HSV (Hue, Saturation, Value)
    lower_blue = np.array([90, 100, 100])
    upper_blue = np.array([130, 255, 255])

    # Create a mask for the blue color
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Apply morphological operations to reduce noise
    kernel = np.ones((7, 7), np.uint8)
    opening_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_OPEN, kernel, iterations=2)
    closing_blue = cv2.morphologyEx(opening_blue, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours in the binary image
    contours_blue, _ = cv2.findContours(closing_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on area in descending order and count the number of tracked objects
    sorted_contours_blue = sorted(contours_blue, key=cv2.contourArea, reverse=True)[:max_tracked_objects_blue]

    # Update the number of tracked objects
    num_tracked_objects_blue = min(len(sorted_contours_blue), max_tracked_objects_blue)

    # Calculate the average centroid value
    for i, contour_blue in enumerate(sorted_contours_blue):

        M_blue = cv2.moments(contour_blue)
        if M_blue["m00"] != 0:
            cx_blue = int(M_blue["m10"] / M_blue["m00"])
            cy_blue = int(M_blue["m01"] / M_blue["m00"])
        else:
            cx_blue, cy_blue = 0, 0

        if i < max_tracked_objects_blue:
            avg_cx_blue[i] += cx_blue
            avg_cy_blue[i] += cy_blue

        # Draw a dot at the centroid
        cv2.circle(frame, (cx_blue, cy_blue), 2, (255, 0, 0), -1)

        # Draw the bounding box
        x, y, w, h = cv2.boundingRect(contour_blue)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

         # Add the label "Blue Object" inside the bounding box
        label = "Goal".format(cx_blue, cy_blue)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

    # Counter for average centroid
    blue_avg_count += 1

    if blue_avg_count == num_frames_average_blue:
        for i in range(num_tracked_objects_blue):
            avg_cx_blue[i] = round(avg_cx_blue[i] / num_frames_average_blue, 2)
            avg_cy_blue[i] = round(avg_cy_blue[i] / num_frames_average_blue, 2)
            print("Goal_Center {}: ({}, {})".format(i+1, avg_cx_blue[i], avg_cy_blue[i]))

        # Reset the accumulated values and counter for the next set of frames
        blue_avg_count = 0
        avg_cx_blue = [0] * max_tracked_objects_blue
        avg_cy_blue = [0] * max_tracked_objects_blue


    return frame

## Thymio Detection (Red)

In [4]:
# Initialize variables for averaging
num_frames_average_red = 30   # Adjust this value
max_tracked_objects_red = 2   # Adjust this value

# Initializing values
red_avg_count = 0
avg_cx_red = [0] * max_tracked_objects_red
avg_cy_red = [0] * max_tracked_objects_red
num_tracked_objects_red = 0

def track_red(frame):
    global red_avg_count, avg_cx_red, avg_cy_red, num_tracked_objects_red

    ## ----------Detecting Red Objects----------
    
    # Convert the frame from BGR to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the red color
    # defined as HSV (Hue, Saturation, Value)
    lower_red = np.array([0, 80, 120])
    upper_red = np.array([10, 255, 255])

    # Create a mask for the red color
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # Apply morphological operations to reduce noise
    kernel = np.ones((5, 5), np.uint8)
    opening_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel, iterations=1)
    closing_red = cv2.morphologyEx(opening_red, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Find contours in the binary image
    contours_red, _ = cv2.findContours(closing_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on area in descending order and count the number of tracked objects
    sorted_contours_red = sorted(contours_red, key=cv2.contourArea, reverse=True) #[:max_tracked_objects_red]

    # Update the number of tracked objects
    num_tracked_objects_red = min(len(sorted_contours_red), max_tracked_objects_red)

    # Calculate the average centroid value
    for i, contour_red in enumerate(sorted_contours_red):

        M_red = cv2.moments(contour_red)
        if M_red["m00"] != 0:
            cx_red = int(M_red["m10"] / M_red["m00"])
            cy_red = int(M_red["m01"] / M_red["m00"])
        else:
            cx_red, cy_red = 0, 0

        if i < max_tracked_objects_red:
            avg_cx_red[i] += cx_red
            avg_cy_red[i] += cy_red

        # Draw a dot at the centroid
        cv2.circle(frame, (cx_red, cy_red), 2, (0, 0, 255), -1)

        # Draw the bounding box
        x, y, w, h = cv2.boundingRect(contour_red)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

         # Add the label "Red Object" inside the bounding box
        label = "T".format(cx_red, cy_red)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    # Counter for average centroid
    red_avg_count += 1

    if red_avg_count == num_frames_average_red:
        for i in range(num_tracked_objects_red):
            avg_cx_red[i] = round(avg_cx_red[i] / num_frames_average_red, 2)
            avg_cy_red[i] = round(avg_cy_red[i] / num_frames_average_red, 2)
            print("Thymio {}: ({}, {})".format(i+1, avg_cx_red[i], avg_cy_red[i]))

        # Reset the accumulated values and counter for the next set of frames
        red_avg_count = 0
        avg_cx_red = [0] * max_tracked_objects_red
        avg_cy_red = [0] * max_tracked_objects_red
            


    return frame


## Map Edge Detection (Green)

In [5]:
# # Initialize variables for averaging
# num_frames_average_green = 30   # Adjust this value
# max_tracked_objects_green = 5   # Adjust this value

# # Initializing values
# green_avg_count = 0
# avg_cx_green = [0] * max_tracked_objects_green
# avg_cy_green = [0] * max_tracked_objects_green
# num_tracked_objects_green = 0

# def track_green(frame):
#     global green_avg_count, avg_cx_green, avg_cy_green, num_tracked_objects_green

#     ## ----------Detecting Green Objects----------
    
#     # Convert the frame from BGR to the HSV color space
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#     # Define the lower and upper bounds for the green color
#     # defined as HSV (Hue, Saturation, Value)
#     lower_green = np.array([10, 40, 10])
#     upper_green = np.array([80, 255, 255])

#     # Create a mask for the green color
#     mask_green = cv2.inRange(hsv, lower_green, upper_green)

#     # Apply morphological operations to reduce noise
#     kernel = np.ones((7, 7), np.uint8)
#     opening_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel, iterations=1)
#     closing_green = cv2.morphologyEx(opening_green, cv2.MORPH_CLOSE, kernel, iterations=1)

#     # Find contours in the binary image
#     contours_green, _ = cv2.findContours(closing_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # Sort contours based on area in descending order and count the number of tracked objects
#     sorted_contours_green = sorted(contours_green, key=cv2.contourArea, reverse=True)[:max_tracked_objects_green]

#     # Update the number of tracked objects
#     num_tracked_objects_green = min(len(sorted_contours_green), max_tracked_objects_green)

#     # Calculate the average centroid value
#     for i, contour_green in enumerate(sorted_contours_green):

#         M_green = cv2.moments(contour_green)
#         if M_green["m00"] != 0:
#             cx_green = int(M_green["m10"] / M_green["m00"])
#             cy_green = int(M_green["m01"] / M_green["m00"])
#         else:
#             cx_green, cy_green = 0, 0

#         if i < max_tracked_objects_green:
#             avg_cx_green[i] += cx_green
#             avg_cy_green[i] += cy_green

#         # Draw a dot at the centroid
#         cv2.circle(frame, (cx_green, cy_green), 2, (0, 0, 0), -1)

#         # Draw the bounding box
#         x, y, w, h = cv2.boundingRect(contour_green)
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 2)

#          # Add the label "Green Object" inside the bounding box
#         label = "Map edge".format(cx_green, cy_green)
#         cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

#     # Counter for average centroid
#     green_avg_count += 1

#     if green_avg_count == num_frames_average_green:
#         for i in range(num_tracked_objects_green):
#             avg_cx_green[i] = round(avg_cx_green[i] / num_frames_average_green, 1)
#             avg_cy_green[i] = round(avg_cy_green[i] / num_frames_average_green, 1)
#             print("Map_Edge_Center {}: ({}, {})".format(i+1, avg_cx_green[i], avg_cy_green[i]))

#         # Reset the accumulated values and counter for the next set of frames
#         green_avg_count = 0
#         avg_cx_green = [0] * max_tracked_objects_green
#         avg_cy_green = [0] * max_tracked_objects_green
            


#     return frame


In [6]:
# Initialize variables for averaging
num_frames_average_green = 100   # Adjust this value
max_tracked_objects_green = 4    # Limit green tracking to 4 objects

# Initializing values
green_avg_count = 0
avg_cx_green = [0] * max_tracked_objects_green
avg_cy_green = [0] * max_tracked_objects_green
num_tracked_objects_green = 0

def assign_coordinates(frame_width, frame_height):
    coordinates = []

    # Assign coordinates based on pixel size
    for i in range(num_tracked_objects_green):
        x_ratio = avg_cx_green[i] / frame_width
        y_ratio = avg_cy_green[i] / frame_height

        if x_ratio < 0.5 and y_ratio < 0.5:
            coordinates.append(("top_left", (avg_cx_green[i], avg_cy_green[i])))
        elif x_ratio >= 0.5 and y_ratio < 0.5:
            coordinates.append(("top_right", (avg_cx_green[i], avg_cy_green[i])))
        elif x_ratio < 0.5 and y_ratio >= 0.5:
            coordinates.append(("bottom_left", (avg_cx_green[i], avg_cy_green[i])))
        else:
            coordinates.append(("bottom_right", (avg_cx_green[i], avg_cy_green[i])))

    return coordinates

def track_green(frame):
    global green_avg_count, avg_cx_green, avg_cy_green, num_tracked_objects_green

    # Convert the frame from BGR to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the green color
    lower_green = np.array([40, 40, 40])  # Adjust these values based on your specific shade of green
    upper_green = np.array([80, 255, 255])

    # Create a mask for the green color
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    # Apply morphological operations to reduce noise
    kernel = np.ones((7, 7), np.uint8)
    opening_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel, iterations=2)
    closing_green = cv2.morphologyEx(opening_green, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours in the binary image
    contours_green, _ = cv2.findContours(closing_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on area in descending order and count the number of tracked objects
    sorted_contours_green = sorted(contours_green, key=cv2.contourArea, reverse=True)[:max_tracked_objects_green]

    # Update the number of tracked objects
    num_tracked_objects_green = min(len(sorted_contours_green), max_tracked_objects_green)

    # Calculate the average centroid value
    for i, contour_green in enumerate(sorted_contours_green):
        M_green = cv2.moments(contour_green)
        if M_green["m00"] != 0:
            cx_green = int(M_green["m10"] / M_green["m00"])
            cy_green = int(M_green["m01"] / M_green["m00"])
        else:
            cx_green, cy_green = 0, 0

        if i < max_tracked_objects_green:
            avg_cx_green[i] = round(avg_cx_green[i] * 0.9 + cx_green * 0.1, 2)  # Apply exponential moving average
            avg_cy_green[i] = round(avg_cy_green[i] * 0.9 + cy_green * 0.1, 2)

        # Draw a dot at the centroid
        cv2.circle(frame, (cx_green, cy_green), 2, (0, 255, 0), -1)

        # Draw the bounding box
        x, y, w, h = cv2.boundingRect(contour_green)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Add the label "Green Object" inside the bounding box
        label = "Green Object {}: ({}, {})".format(i+1, avg_cx_green[i], avg_cy_green[i])
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

    # Counter for average centroid
    green_avg_count += 1

    if green_avg_count == num_frames_average_green:
        for i in range(num_tracked_objects_green):
            print("Green Object {}: ({}, {})".format(i+1, avg_cx_green[i], avg_cy_green[i]))

        # Assign and print coordinates based on pixel size
        frame_width = frame.shape[1]
        frame_height = frame.shape[0]
        coordinates = assign_coordinates(frame_width, frame_height)
        for coord in coordinates:
            # Assuming coord is a tuple of length 3
            if len(coord) >= 3:
                print("Coordinate {}: {}".format(coord[0], (coord[1], coord[2])))
            else:
                print("Invalid coordinate format: {}".format(coord))

        # Reset the accumulated values and counter for the next set of frames
        green_avg_count = 0
        avg_cx_green = [0] * max_tracked_objects_green
        avg_cy_green = [0] * max_tracked_objects_green

    return frame

## Perspective Transformation

In [7]:
def change_perspective(frame, centroids):
    # Define the source and destination points for perspective transformation
    src_points = np.float32(centroids)
    dst_points = np.float32([[0, 0], [300, 0], [0, 300], [300, 300]])

    # Get the perspective transformation matrix
    M = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply the perspective transformation to the frame
    transformed_frame = cv2.warpPerspective(frame, M, (300, 300))

    return transformed_frame

## Main Function

In [8]:
def main():
    # Get the video file and read it
    # video_path = 'map.mp4'
    video = cv2.VideoCapture(1)
    ret, frame = video.read()

    frame_height, frame_width = frame.shape[:2]
    # Resize the video for a more convenient view
    frame = cv2.resize(frame, [frame_width // 2, frame_height // 2])

    frame_count = 0
    
    while True:
        # Record the start time for calculating processing time
        start_time = time.time()

        # Read a frame from the webcam
        ret, frame = video.read()
        frame = cv2.resize(frame, [frame_width // 2, frame_height // 2])

        if not ret:
            print("Failed to capture frame")
            break

        # Call track_black func and enhance black color in the frame and draw bounding boxes
        track_frame = track_black(frame)  # Create a copy to preserve the original frame

        # Call track_blue func and enhance blue color in the frame and draw bounding boxes
        track_frame = track_blue(frame)  # Create a copy to preserve the original frame

        track_frame = track_red(frame)
        track_frame = track_green(frame)


        # Display the original frame with bounding boxes for black and blue objects
        cv2.imshow('Object Tracking', track_frame)

        # Break the loop if the user presses 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Obstacle_Center 1: (121.0, 111.0)
Obstacle_Center 2: (206.0, 73.0)
Obstacle_Center 3: (172.07, 153.0)
Goal_Center 1: (5.5, 4.7)
Thymio 1: (174.0, 23.1)
Obstacle_Center 1: (121.0, 111.0)
Obstacle_Center 2: (206.0, 73.0)
Obstacle_Center 3: (172.0, 153.0)
Goal_Center 1: (170.23, 148.0)
Thymio 1: (174.03, 23.4)
Obstacle_Center 1: (121.0, 111.0)
Obstacle_Center 2: (206.0, 73.0)
Obstacle_Center 3: (172.33, 153.0)
Goal_Center 1: (170.3, 150.5)
Thymio 1: (174.07, 23.4)
Green Object 1: (252.96, 197.96)
Green Object 2: (68.96, 195.91)
Green Object 3: (241.9, 30.87)
Green Object 4: (103.14, 20.96)
Invalid coordinate format: ('bottom_right', (252.96, 197.96))
Invalid coordinate format: ('bottom_left', (68.96, 195.91))
Invalid coordinate format: ('top_right', (241.9, 30.87))
Invalid coordinate format: ('top_left', (103.14, 20.96))
Obstacle_Center 1: (122.33, 110.4)
Obstacle_Center 2: (204.9, 75.67)
Obstacle_Center 3: (167.13, 147.9)
Goal_Center 1: (171.2, 151.0)
Thymio 1: (174.13, 23.4)
Obstacle_Ce

KeyboardInterrupt: 